<a href="https://colab.research.google.com/github/chanukaDissanayaka/RideFareClassification/blob/master/Ridefarev3_submission_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from datetime import datetime, timedelta
from pandas import DataFrame

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

import numpy as np

RANDOM_SEED = 4

Using TensorFlow backend.


In [0]:
dataframe = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [0]:
dataframe.isna().sum(axis = 0)
dataframe = dataframe.fillna(method='ffill')

In [0]:
train_set = format_train_Data(dataframe)
labels = get_labels(dataframe)

In [0]:
train_set.shape

(17176, 20)

In [0]:
labels.shape

(17176,)

In [0]:
for i in labels:
  print(i)

In [0]:
train_set.keys()

Split

In [0]:
X_train, X_eval, Y_train, Y_eval = train_test_split(
    train_set,
    labels,
    test_size=0.20,
    shuffle=True,
    stratify=labels,
    random_state=RANDOM_SEED
)

In [0]:
test_data = format_test_Data(test)
test_data.shape

(8576, 20)

TPOT


In [0]:
tpot = TPOTClassifier() 
tpot.fit(X_train, Y_train)
tpot.score(X_eval, Y_eval)

Model

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=50, random_state=0)
rf.fit(X_train, Y_train) 
rf.score(X_eval, Y_eval)     

0.9275320139697323

Test Data and predict

In [0]:
preds = rf.predict(test_data)


8138 438 acc :  0.9489272388059702


OUTPUT

In [0]:

output = pd.DataFrame(test,columns=['tripid'])
output['prediction'] = preds
output.head()


,tripid,prediction
0,213284604,1
1,213286352,0
2,213293973,1
3,213294622,1
4,213298687,1


In [0]:
output.to_csv('/content/output_ridefare_2.csv',index=False)

In [0]:
def get_scaler(train_data_frame):
  dataframe = frame
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  scaled_features = features
  scaler = StandardScaler()
  numeric_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 
                'meter_waiting_till_pickup', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare']

  scaled_features[numeric_cols] = scaler.fit_transform(scaled_features[numeric_cols])
  return scaler

In [0]:
def format_train_Data(frame):
  dataframe = frame
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  scaled_features = features
  scaler = StandardScaler()
  numeric_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 
                'meter_waiting_till_pickup', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare']

  scaled_features[numeric_cols] = scaler.fit_transform(scaled_features[numeric_cols])

  td = scaled_features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  td_scaled = td

  td_scaled = scaler.fit_transform(td_scaled)
  return td_scaled

def get_labels(dataframe):
  label_set = dataframe.iloc[:, 13:14]

  rev_label_set = label_set.replace({'correct':'incorrect', 'incorrect':'correct'})

  encoder = LabelEncoder()
  encoder.fit(rev_label_set)
  labels = encoder.transform(rev_label_set)
  return labels

In [0]:
def format_test_Data(frame):
  dataframe = frame
  feature_set = dataframe.iloc[:,1:13]
  label_set = dataframe.iloc[:, 13:14]

  #encoder = LabelEncoder()
  #encoder.fit(label_set)
  #labels = encoder.transform(label_set)

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  scaled_features = features
  scaler = StandardScaler()
  numeric_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 
                'meter_waiting_till_pickup', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare']

  scaled_features[numeric_cols] = scaler.fit_transform(scaled_features[numeric_cols])

  td = scaled_features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  td_scaled = td

  scaler = StandardScaler()

  td_scaled = scaler.fit_transform(td_scaled)
  return td_scaled